In [1]:
%load_ext autoreload
%autoreload 2


from hangar import Repository

import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

# Local Setup

In [2]:
repo = Repository('/Users/rick/projects/tensorwerk/hangar/dev/dota/')

HANGAR RUNTIME WARNING: no repository exists at /Users/rick/projects/tensorwerk/hangar/dev/dota/__hangar, please use `init_repo` function


In [3]:
repo.init(user_name='Rick Izzo', user_email='rick@tensorwerk.com', remove_old=True)
co = repo.checkout(write=True)

Hangar Repo initialized at: /Users/rick/projects/tensorwerk/hangar/dev/dota/__hangar


In [4]:
testData = np.loadtxt('/Users/rick/projects/tensorwerk/hangar/dev/data/dota2Dataset/dota2Test.csv', delimiter=',', dtype=np.uint8)
trainData = np.loadtxt('/Users/rick/projects/tensorwerk/hangar/dev/data/dota2Dataset/dota2Train.csv', delimiter=',', dtype=np.uint16)

In [5]:
testName = 'test'
testPrototype = testData[0]
trainName = 'train'
trainPrototype = trainData[0]

In [6]:
co.datasets.init_dataset(name=testName, prototype=testPrototype, samples_are_named=False)

testDset = co.datasets[testName]
pbar = tqdm(total=testData.shape[0])
with testDset as ds:
    for gameIdx, gameData in enumerate(testData):
        if (gameIdx % 500 == 0):
            pbar.update(500)
        ds.add(gameData)
pbar.close()

co.commit('initial commit on master with test data')

Dataset Specification:: Name: `test`, Initialization style: `prototype`, Shape: `(117,)`, DType: `uint8`, Samples Named: `False`, Variable Shape: `False`, Max Shape: `(117,)`
Dataset Initialized: `test`



Commit operation requested with message: initial commit on master with test data
(823736, 340510, 65536)
removing all stage hash records
Commit completed. Commit hash: 829210950e8b8d0efcbfc8842c61497b11766cf4


'829210950e8b8d0efcbfc8842c61497b11766cf4'

In [7]:
repo.create_branch('add-train')

co.close()

writer checkout of master closed


In [8]:
co = repo.checkout(write=True, branch_name='add-train')

co.datasets.init_dataset(name=trainName, prototype=trainPrototype, samples_are_named=False)
trainDset = co.datasets[trainName]

pbar = tqdm(total=trainData.shape[0])
with trainDset as dt:
    for gameIdx, gameData in enumerate(trainData):
        if (gameIdx % 500 == 0):
            pbar.update(500)
        dt.add(gameData)
pbar.close()

co.commit('added training data on another branch')
co.close()

Dataset Specification:: Name: `train`, Initialization style: `prototype`, Shape: `(117,)`, DType: `uint16`, Samples Named: `False`, Variable Shape: `False`, Max Shape: `(117,)`
Dataset Initialized: `train`



Commit operation requested with message: added training data on another branch
(8328595, 3405102, 65536)
removing all stage hash records
Commit completed. Commit hash: 06d4bffed975ce2ccd4f89ea6bd628df64ddc18d
writer checkout of add-train closed


In [9]:
repo.log()

* 06d4bffed975ce2ccd4f89ea6bd628df64ddc18d (add-train) : added training data on another branch
* 829210950e8b8d0efcbfc8842c61497b11766cf4 (master) : initial commit on master with test data


# Push to remote

server running on google cloud at `35.227.114.110:50051`

In [10]:
repo.add_remote('gcloud', '35.227.114.110:50051')

True

### Push the master branch 

In [11]:
repo.push('gcloud', 'master')

True

### Push the training branch

In [12]:
repo.push('gcloud', 'add-train')

True

### Now Restart and See How this Works!

`hangar clone 35.227.114.110:50051`

In [2]:
repo = Repository('/Users/rick/projects/tensorwerk/hangar/dev/dota-clone/')

In [3]:
repo.list_branch_names()

['master']

In [4]:
repo.log()

* 829210950e8b8d0efcbfc8842c61497b11766cf4 (master) : initial commit on master with test data


In [5]:
repo.fetch('origin', 'add-train')

'origin/add-train'

In [ ]:
repo.list_branch_names()

In [ ]:
repo.log(branch_name='origin/add-train')

# merge the commits and push!

In [ ]:
co = repo.checkout(write=True, branch_name='master')
co.reset_staging_area()

In [ ]:
repo.merge('ff merge', 'master', 'origin/add-train')

In [ ]:
repo.log()

In [ ]:
repo.push('origin', 'master')

# Check that it worked!

In [ ]:
repo.fetch('origin', 'master')

In [ ]:
repo.summary()

# Try Cloning in a fresh repo just to be sure!

In [ ]:
repo = Repository('/Users/rick/projects/tensorwerk/hangar/dev/dota-clone-2/')

In [ ]:
repo.clone('Rick Izzo', 'rick@tensorwerk.com', remote_address='35.227.114.110:50051')

In [ ]:
repo.log()

In [ ]:
co = repo.checkout()

In [ ]:
co.datasets

In [ ]:
co.datasets['train']

In [ ]:
repo.summary()